In [39]:
import numpy as np
import pandas as pd
from datetime import datetime
import datetime as dt
from bokeh.io import output_file, show, output_notebook, curdoc, reset_output
from bokeh.plotting import figure
from bokeh.palettes import Category20, inferno
from bokeh.models import FactorRange, Button, ColumnDataSource, Legend, LegendItem, Slider, VBar, Plot, Panel, Select, CustomJS
from bokeh.layouts import column, row, widgetbox, layout
from bokeh.models.widgets import Div
import matplotlib.pyplot as plt
import seaborn as sb
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, Slider, PreText, CheckboxButtonGroup, WheelZoomTool
from bokeh.palettes import Spectral6, brewer
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Paragraph,
                          Legend, LegendItem, Label,FactorRange, Div)
from bokeh.models.widgets import Panel, Tabs
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, STAMEN_TONER
from bokeh.transform import jitter
from bokeh.embed import file_html
from bokeh.resources import CDN
import matplotlib.pyplot as plt

Noter:

Contributing factor graph: Alle crashes årsager som fremgår over 10000 gange i data2 sættet er udvalgt. Herefter er andelen ud af antal crahses der fremgår i data2 det år den måned udregnet. Alle de steder der står NaN er erstattet med 0.  

In [40]:
# Data import 
data2 = pd.read_csv("C:\\Users\Dude\Desktop\Vision Zero\crash_data.csv",low_memory=False)
#data2 = data2.sample(200000)

In [41]:
# Changing formatting in 'Date' and 'Time' columns
data2["CRASH DATE"] = data2["CRASH DATE"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
data2["CRASH TIME"] = data2["CRASH TIME"].apply(lambda x: datetime.strptime(x, "%H:%M:%S"))

In [42]:
# Adding a new column which contains year from CRASH DATE variable
ML_data = data2.copy()
data2['year'] = data2['CRASH DATE'].dt.year
data2['month'] = data2['CRASH DATE'].dt.month
data2['hour'] = data2['CRASH TIME'].dt.hour
data2['weekday']= data2['CRASH DATE'].dt.day_name()

df_crash = data2.copy()

# Replacing nan values by 'UNKNOWN' in BOROUGH column
data2.BOROUGH.fillna('UNKNOWN', inplace=True)

# ANDERS

In [43]:
df_crash.drop(df_crash.loc[:, 'CONTRIBUTING FACTOR VEHICLE 1':'VEHICLE TYPE CODE 5'].columns, axis = 1, inplace = True) 
df_crash.drop(df_crash.loc[:, 'ON STREET NAME':'OFF STREET NAME'].columns, axis = 1, inplace=True)
df_crash = df_crash.loc[(df_crash['NUMBER OF PERSONS INJURED'] > 0) | (df_crash['NUMBER OF PERSONS KILLED'] > 0 )]  
df_crash = df_crash.loc[(df_crash['LATITUDE'].notna() & df_crash['LONGITUDE'].notna() & df_crash['LONGITUDE']!=0 & (df_crash['LATITUDE'] != 0 )) & df_crash['BOROUGH'].notna() & (df_crash['LATITUDE']<41)].copy()

borough_colors = inferno(3)

def wgs84_to_web_mercator(df_crash, lon="LONGITUDE", lat="LATITUDE"):
    k = 6378137
    df_crash["x"] = df_crash[lon] * (k * np.pi/180.0)
    df_crash["y"] = np.log(np.tan((90 + df_crash[lat]) * np.pi/360.0)) * k
    return df_crash
wgs84_to_web_mercator(df_crash)

xmin, xmax = min(df_crash['x']), max(df_crash['x'])
ymin, ymax = min(df_crash['y']), max(df_crash['y'])

borough_data = df_crash.groupby([df_crash['year'],df_crash['BOROUGH']]).agg({'NUMBER OF PERSONS KILLED': 'sum','NUMBER OF PERSONS INJURED':'sum', 
                                                                            'NUMBER OF PEDESTRIANS KILLED':'sum','NUMBER OF CYCLIST KILLED':'sum',
                                                                            'NUMBER OF MOTORIST KILLED':'sum','NUMBER OF PEDESTRIANS INJURED':'sum',
                                                                            'NUMBER OF CYCLIST INJURED':'sum','NUMBER OF MOTORIST INJURED':'sum'}).reset_index()
borough_data['PEDESTRIANS'] = borough_data['NUMBER OF PEDESTRIANS KILLED'] + borough_data['NUMBER OF PEDESTRIANS INJURED']
borough_data['CYCLISTS'] = borough_data['NUMBER OF CYCLIST KILLED'] + borough_data['NUMBER OF CYCLIST INJURED']
borough_data['MOTORISTS'] = borough_data['NUMBER OF MOTORIST KILLED'] + borough_data['NUMBER OF MOTORIST INJURED']
borough_data['KILLED'] = borough_data['NUMBER OF PEDESTRIANS KILLED'] + borough_data['NUMBER OF CYCLIST KILLED'] + borough_data['NUMBER OF MOTORIST KILLED']  
borough_data['INJURED'] = borough_data['NUMBER OF PEDESTRIANS INJURED'] + borough_data['NUMBER OF CYCLIST INJURED'] + borough_data['NUMBER OF MOTORIST INJURED']

df_crash2 = df_crash[['x','y','CRASH TIME','weekday']]
source2_vbar = ColumnDataSource(data =borough_data.sample(0))
source3_vbar = ColumnDataSource(data =borough_data)
source_map = ColumnDataSource(data = df_crash2.sample(0))
data_source_map = ColumnDataSource(data = df_crash)

#PLOT OF MAP 
plot_map = figure(plot_height = 600, plot_width =650,x_axis_type="mercator", y_axis_type="mercator", toolbar_location = "left",x_range = (xmin-1000,xmax+1000),y_range=(ymin-1000,ymax+1000))
plot_map.axis.visible = False

tile_provider = get_provider(CARTODBPOSITRON)
plot_map.add_tile(tile_provider)
plot_map.circle(x='x', y='y', fill_alpha = 0.05, line_alpha = 0, source = source_map, color = 'red', size=6)
plot_map.toolbar.active_scroll = plot_map.select_one(WheelZoomTool)

#PLOT VBAR_STACK
boroughs = df_crash['BOROUGH'].unique()
road_user = ['PEDESTRIANS','CYCLISTS','MOTORISTS']
p_borough = figure(x_range = boroughs,plot_width=300, y_range = ((0,20000)), plot_height=580, tools = "hover", tooltips = "$name: @$name", title="Distribution of road users")
p_borough.vbar_stack(road_user, x='BOROUGH', width=0.4, color=borough_colors, source=source2_vbar, legend_label =road_user);
p_borough.xaxis.major_label_orientation = 'vertical'
p_borough.legend.label_text_font_size = '7pt'
p_borough.legend.padding = 0
#DEATH/INJURY COUNT
div_injuries = Div(text="""<font size='5'>   Injuries: 0</font>""",
                   width=200, height=30)
div_deaths = Div(text="""<font size='5'>   Deaths: 0 </font>""",                
                    width=200, height=30)

#Jitterplot - Time
DAYS = ['Sunday', 'Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday']
p2_hour_jitter = figure(plot_width=950, plot_height=250, y_range=DAYS, x_axis_type='datetime', 
           title="Weekday and time of incidents")

p2_hour_jitter.circle(x='CRASH TIME', y=jitter('weekday', width=0.7, range=p2_hour_jitter.y_range),  source=source_map, alpha=0.1)

p2_hour_jitter.xaxis[0].formatter.days = ['%H:%M']
p2_hour_jitter.x_range.range_padding = 0
p2_hour_jitter.ygrid.grid_line_color = None

#On Change


In [44]:
#MACHINE LEARNING

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler

ML= ML_data[['CRASH DATE','CRASH TIME','BOROUGH','CONTRIBUTING FACTOR VEHICLE 1']]
ML2 = ML.loc[ML['CONTRIBUTING FACTOR VEHICLE 1'].isin(['Alcohol Involvement','Pavement Slippery'])].copy()
ML2['CRASH DATE'] = ML2['CRASH DATE'].dt.year
ML2['CRASH TIME'] = ML2['CRASH TIME'].dt.hour
ML2['25mph'] = 0
ML2.loc[(ML2['CRASH DATE'] != 2013) & (ML2['CRASH DATE'] != 2014),'25mph'] = 1 
ML2.rename(columns={'CRASH DATE':'Year','CRASH TIME':'Hour','CONTRIBUTING FACTOR VEHICLE 1':'Reason','BOROUGH':'Borough',},inplace=True)
ML2 = ML2.loc[ML2.Borough.notna()].copy()
ML2.drop(['Year'],axis=1,inplace=True)

crash_ML = pd.concat((ML2,pd.get_dummies(ML2.Borough)),1)
crash_ML.drop(['Borough'],axis=1,inplace=True)
crash_ML.columns = map(str.capitalize, crash_ML.columns)


#Splitting data into features (X) and labels (y)
X = crash_ML.drop(['Reason'],axis=1)
y = crash_ML['Reason']

#Randomly choosing 20% of the data as our test data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

#fitting our Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train);
pred_rfc=rfc.predict(X_test)

combs = ML2.groupby(['Hour','Borough','25mph']).size().reset_index()
cols = ['Hour', '25mph', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten island','Reason']
crash2_ML = crash_ML.copy()
crash2_ML=crash2_ML[cols].copy()

#Reading predictions into DF
crash2_ML['predictions'] = rfc.predict(crash2_ML.iloc[:,:-1].values)
crash2_ML.drop(['Reason'],axis=1,inplace = True)
crash2_ML.drop_duplicates(inplace=True)

div_deaths = Div(text="""<font size='5'>   Deaths: 0 </font>""",                
                    width=200, height=30)
data_ML_source = ColumnDataSource(data = crash2_ML)

In [45]:
# Liste over alle columns, man skal kunne vælge imellem:
hour_list = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
borough_list = ['Bronx','Brooklyn','Manhattan','Queens','Staten island']
speed_reduction = ['30 mph','25 mph']
select_hour = Select(title="Hour", value='0', options=hour_list)
select_borough =Select(title="Borough:", value='Queens', options=borough_list)
select_speed = Select(title="Speed limit:", value='30 mph', options=speed_reduction)

button_ML = Button(label="Predict cause of accident!")

div_ML = Div(text="""<font size='5.5'>...</font>""",                
                    width=400, height=80)


In [46]:

callbackML = CustomJS(args={'select_hour':select_hour,'select_borough':select_borough,'select_speed':select_speed, 'div_ML':div_ML,'data_ML_source':data_ML_source},
                    code = """
            var data = data_ML_source.data;
            var hour_value = select_hour.value;
            var borough_value = select_borough.value;
            var speed_value = select_speed.value;
            var borough_array = [];
            var div_predict = "";
            var speed_str = ""
            
            div_ML.text="<font size='5'>"+"..."+"</font>";
            
            setTimeout(myfunc,1000);
            
            function myfunc() {
            // data['Hour'].length
            if (speed_value == '25 mph') {
                speed_str = '1'
            } else {
                speed_str = '0'
            }
            for(var i=0;i<data['Hour'].length;i++){
                if (data['Hour'][i] == hour_value && data['25mph'][i] == speed_str) {
                    if (data[borough_value][i]==1) {
                        div_predict = data['predictions'][i]
                    }                    
                }
            }
            div_ML.text="<font size='5.5'>"+div_predict+"</font>";
            }
""")
               

In [47]:
#Slider & Checkboxs
checkbox_labels = ['Injuries','Deaths']
checkbox_button_group = CheckboxButtonGroup(
        labels=checkbox_labels, active=[])

slider = Slider(start=2013, end = 2019, step = 1, value = 2013, title = 'Year')
callback = CustomJS(args={'source_map': source_map, 'data_source_map':data_source_map, 'source2_vbar': source2_vbar, 'source3_vbar': source3_vbar, 
                          'y':p_borough.y_range, 'checkbox_labels':checkbox_labels, 
                          'checkbox_button_group':checkbox_button_group, 'slider':slider, 'div_injuries':div_injuries,'div_deaths':div_deaths},
                    code = """
    var data_s = data_source_map.data;
    var data3 = source3_vbar.data;
    var slival = slider.value
    let selected_categories = checkbox_button_group.active.map((i) => checkbox_labels[i])
    var test = [];
    var test2 = [];
    test['x']=[];
    test['y']=[];
    test['CRASH TIME']=[];
    test['weekday']=[];
    test2['PEDESTRIANS']=[];
    test2['BOROUGH']=[];
    test2['CYCLISTS']=[];
    test2['MOTORISTS']=[];
    test2['KILLED']=[];
    test2['INJURED']=[];
    
    function myFunc(total, num) {
        return total+num;
    }

    for(var i=0;i<data_s['CRASH TIME'].length;i++){
        if(data_s["year"][i] == slival) {
            if (selected_categories.indexOf("Injuries")>-1 && selected_categories.indexOf("Deaths")>-1) {
                 if(data_s['NUMBER OF PERSONS INJURED'][i]>0 || data_s['NUMBER OF PERSONS KILLED'][i]>0){                
                     test['x'].push(data_s['x'][i]);
                     test['y'].push(data_s['y'][i]);
                     test['CRASH TIME'].push(data_s['CRASH TIME'][i]);
                     test['weekday'].push(data_s['weekday'][i]);
                 }
            } else if(selected_categories.indexOf("Injuries")>-1) {
                 if(data_s['NUMBER OF PERSONS INJURED'][i]>0){                
                     test['x'].push(data_s['x'][i]);
                     test['y'].push(data_s['y'][i]);
                     test['CRASH TIME'].push(data_s['CRASH TIME'][i]);
                     test['weekday'].push(data_s['weekday'][i]);
                 }
            } else if(selected_categories.indexOf("Deaths")>-1){
                 if(data_s['NUMBER OF PERSONS KILLED'][i]>0){                
                     test['x'].push(data_s['x'][i]);
                     test['y'].push(data_s['y'][i]);
                     test['CRASH TIME'].push(data_s['CRASH TIME'][i]);
                     test['weekday'].push(data_s['weekday'][i]);
                 }
            }
                    
        }
    }
            
    for(var i=0;i<data3['year'].length;i++){
         if(data3["year"][i] == slival) {
             if (selected_categories.indexOf("Injuries")>-1 && selected_categories.indexOf("Deaths")>-1) {             
                test2['PEDESTRIANS'].push(data3['PEDESTRIANS'][i]);
                test2['BOROUGH'].push(data3['BOROUGH'][i]);
                test2['CYCLISTS'].push(data3['CYCLISTS'][i]);
                test2['MOTORISTS'].push(data3['MOTORISTS'][i]);
                test2['KILLED'].push(data3['KILLED'][i]);
                test2['INJURED'].push(data3['INJURED'][i]);             
                y.end = Math.max(...data3['INJURED'])+Math.max(...data3['KILLED']);                
            } else if(selected_categories.indexOf("Injuries")>-1){            
                test2['PEDESTRIANS'].push(data3['NUMBER OF PEDESTRIANS INJURED'][i]);
                test2['BOROUGH'].push(data3['BOROUGH'][i]);
                test2['CYCLISTS'].push(data3['NUMBER OF CYCLIST INJURED'][i]);
                test2['MOTORISTS'].push(data3['NUMBER OF MOTORIST INJURED'][i]);
                test2['KILLED'].push(data3['KILLED'][i]);
                test2['INJURED'].push(data3['INJURED'][i]);
                y.end = Math.max(...data3['INJURED']);
            } else if(selected_categories.indexOf("Deaths")>-1) {
                test2['PEDESTRIANS'].push(data3['NUMBER OF PEDESTRIANS KILLED'][i]);
                test2['BOROUGH'].push(data3['BOROUGH'][i]);
                test2['CYCLISTS'].push(data3['NUMBER OF CYCLIST KILLED'][i]);
                test2['MOTORISTS'].push(data3['NUMBER OF MOTORIST KILLED'][i]);
                test2['KILLED'].push(data3['KILLED'][i]);
                test2['INJURED'].push(data3['INJURED'][i]);
                y.end = Math.max(...data3['KILLED']);
            }
        } 
    }
    
    if (selected_categories.indexOf("Injuries")>-1 && selected_categories.indexOf("Deaths")>-1) {
        div_injuries.text="<font size='5'>"+'   Injuries: ' + test2['INJURED'].reduce(myFunc) +"</font>";
        div_deaths.text="<font size='5'>"+'   Deaths: ' + test2['KILLED'].reduce(myFunc) +"</font>";
    } else if(selected_categories.indexOf("Injuries")>-1){
        div_injuries.text="<font size='5'>"+'   Injuries: ' + test2['INJURED'].reduce(myFunc) +"</font>";
        div_deaths.text="<font size='5'>"+'   Deaths: 0'+"</font>";
    } else if(selected_categories.indexOf("Deaths")>-1) {
        div_injuries.text="<font size='5'>"+'   Injuries: 0'+"</font>";
        div_deaths.text="<font size='5'>"+'   Deaths: ' + test2['KILLED'].reduce(myFunc)+"</font>";
    } else {
        div_injuries.text="<font size='5'>"+'   Injuries: 0'+"</font>";
        div_deaths.text="<font size='5'>"+'   Deaths: 0'+"</font>";
    }
    //y.end = Math.max(...test2['INJURED']);
    source_map.data = test;
    source2_vbar.data = test2;
    source_map.change.emit();
    source2_vbar.change.emit();
""")

   
                    





In [48]:
checkbox_button_group.js_on_change('active', callback)    
slider.js_on_change('value', callback)
#select_hour.js_on_change('value', callbackML)
#select_borough.js_on_change('value', callbackML)
#select_speed.js_on_change('value', callbackML)
button_ML.js_on_click(callbackML)
#Layout
# plots_yaaa = column(slider,checkbox_button_group,div,p_borough)
# l1 = layout([
#     [plots_yaaa,plot_map],
#     [p2_hour_jitter]
# ])

#plioo = column(div,p_borough)

injur_slider = column(row(checkbox_button_group,slider),plot_map)
div_comb = column(div_injuries,div_deaths)
div_vstack = column(div_comb,p_borough)
l1 = layout([
    [injur_slider,div_vstack],
    [p2_hour_jitter]
])

l2 = layout([[column(row(select_hour,select_borough,select_speed,width = 600),div_ML)],[button_ML]])
tab3 = Panel(child=l1, title="Julie giver kage .. fuckfuck lort ")
tab4 = Panel(child=l2, title= 'Machine Learning')
#tabs = Tabs(tabs=[tab1])
#curdoc().add_root(tabs)
#curdoc().title ="Sliders"
#show(tabs)
#output_file("js_events.html", title="Fuck2")

# Stacked bar chart - Number of collisions per year and borough

In [49]:
borough =['BROOKLYN', 'QUEENS', 'MANHATTAN', 'UNKNOWN', 'BRONX',
       'STATEN ISLAND']

years = ['{}'.format(i) for i in data2['CRASH DATE'].dt.year.sort_values(ascending=True).unique()]

data2['year_string'] = data2['CRASH DATE'].dt.year.astype(str)
crash_data_p11 = data2.groupby(['year_string','BOROUGH']).size().unstack().copy()
source_p11 = ColumnDataSource(crash_data_p11)

colors = inferno(6)

p_11 = figure(x_range=FactorRange(*years), y_range= (0,250000), plot_height=300, plot_width=600, title="Collisions per borough over year",
           x_axis_label = 'Year', y_axis_label = 'Number of collisions',
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

p_11.title.text_font_size = '12pt'
p_11.xaxis.axis_label_text_font_size = "8pt"
p_11.yaxis.axis_label_text_font_size = "8pt"
bar ={} 
items = []
v = p_11.vbar_stack(borough, x='year_string', width=0.8, color=colors, source=source_p11, muted_alpha= 0.1, muted=False)

legend = Legend(items=[(x, [v[i]]) for i, x in enumerate(borough)], location=(0, 30))
p_11.add_layout(legend, 'left')
p_11.legend.click_policy="mute" 
p_11.y_range.start = 0
p_11.x_range.range_padding = 0.1
p_11.xgrid.grid_line_color = None
p_11.axis.minor_tick_line_color = None
p_11.outline_line_color = None


In [50]:
output_notebook()

Loading BokehJS ...

# Stacked bar chart - weekdays

In [52]:
colors = inferno(6)

borough =['BROOKLYN', 'QUEENS', 'MANHATTAN', 'UNKNOWN', 'BRONX','STATEN ISLAND']
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

crash_data_p21 = data2.loc[data2['year']== 2013].groupby(['year', 'weekday','BOROUGH']).size().unstack().reset_index().copy()
crash_data_p21_real = data2.groupby(['year','weekday','BOROUGH']).size().unstack().reset_index()
source_p21 = ColumnDataSource(data=crash_data_p21)
source_p21_real = ColumnDataSource(data=crash_data_p21_real)

bar ={} 
items = []

p_21 = figure(x_range=FactorRange(*weekdays), y_range= (0,40000), plot_height=300, plot_width=600, title="Collisions per borough over weekdays",
           x_axis_label = 'Weekday', y_axis_label = 'Number of collisions',
           toolbar_location=None, tools="hover", tooltips="$name: @$name")

p_21.title.text_font_size = '12pt'
p_21.xaxis.axis_label_text_font_size = "8pt"
p_21.yaxis.axis_label_text_font_size = "8pt"

v = p_21.vbar_stack(borough, x='weekday', width=0.8, color=colors, source=source_p21)
legend = Legend(items=[(x, [v[i]]) for i, x in enumerate(borough)], location=(0, 30))
p_21.add_layout(legend, 'left')

p_21.y_range.start = 0
p_21.x_range.range_padding = 0.1
p_21.xgrid.grid_line_color = None
p_21.axis.minor_tick_line_color = None
p_21.outline_line_color = None
p_21.xaxis.major_label_orientation = 'vertical'
show(p_21)

# Graph collisions - Weekday

In [53]:
crash_data_p22 = data2.loc[data2['year']==2013].groupby(['hour','BOROUGH']).size().unstack().copy()
crash_data_p22_real = data2.groupby(['year','hour','BOROUGH']).size().unstack().reset_index()

source_p22 = ColumnDataSource(data = crash_data_p22)
source_p22_real = ColumnDataSource(data = crash_data_p22_real)

#legend_list = data2['BOROUGH'].unique()
legend_list = ['BROOKLYN', 'QUEENS', 'MANHATTAN', 'UNKNOWN', 'BRONX','STATEN ISLAND'] 
colors_list = inferno(6)

p_22 = figure(plot_width=600, plot_height=300, title="Collisions per borough over hours",
           x_axis_label = 'Hour', y_axis_label = 'Number of collisions',
           toolbar_location=None, tools="hover", tooltips="$BOROUGH: @BOROUGH")

p_22.title.text_font_size = '12pt'
p_22.xaxis.axis_label_text_font_size = "8pt"
p_22.yaxis.axis_label_text_font_size = "8pt"

legend_it = []

for (indx,i),color in zip(enumerate(legend_list),colors_list):
    my_plot = p_22.line(x='hour', y=i, color= color, line_width=2.0, source=source_p22)
    legend_it.append((i, [my_plot]))
    
legend = Legend(items=legend_it, location=(0, 30))
p_22.add_layout(legend, 'left')


# Graph collisions - Month

In [54]:
#output_notebook()

crash_data_p12 = data2.loc[data2['year']== 2013].groupby(['month','BOROUGH']).size().unstack().copy()
crash_data_p12_real = data2.groupby(['year','month','BOROUGH']).size().unstack().reset_index()

source_p12 = ColumnDataSource(data = crash_data_p12)
source_p12_real = ColumnDataSource(data = crash_data_p12_real)


legend_list = data2['BOROUGH'].unique()
colors_list = inferno(6)

p_12 = figure(plot_width=600, plot_height=300, title="Collisions per borough over months",
           x_axis_label = 'Month', y_axis_label = 'Number of collisions',
           toolbar_location=None, tools="hover", tooltips="Brooklyn: @BROOKLYN")


p_12.title.text_font_size = '12pt'
p_12.xaxis.axis_label_text_font_size = "8pt"
p_12.yaxis.axis_label_text_font_size = "8pt"

legend_it = []

for (indx,i),color in zip(enumerate(legend_list),colors_list):
    my_plot = p_12.line(x='month', y=i, color= color, line_width=2.0, source=source_p12)
    legend_it.append((i, [my_plot]))
    
legend = Legend(items=legend_it, location=(0, 30))
p_12.add_layout(legend, 'left')


In [55]:
############################## Text ######################################

div_text = Div(text="""<center><h1><strong><Overview></h1>
                        <p> Her skal der stå noget tekst. PUHAAA. Anders giver kage. </p>""")


# Slider

In [56]:
#slider = Slider(start=2013, end=2019, value=2013, step=1, title= 'Bum')

#def update_plot(attr,old,new):
#    yr=slider.value
#    source_p21.data = data2.loc[data2['year'] == yr].groupby(['year', 'weekday','BOROUGH']).size().unstack().reset_index().copy()
#    source_p12.data = data2.loc[data2['year'] == yr].groupby(['month','BOROUGH']).size().unstack().copy()
#    source_p22.data = data2.loc[data2['year']==yr].groupby(['hour','BOROUGH']).size().unstack().copy()
 
#slider.on_change('value', update_plot)    



#layout = row(column(div_text, p_11, p_21), column(slider,p_12, p_22))
#tab1 = Panel(child = layout, title = 'Overview')


# Plot Fatal collisions per month over year

In [57]:
#output_notebook()

# Dataset contains only fatal collisions: 
data_killed = data2.loc[data2['NUMBER OF PERSONS KILLED'] > 0].groupby(['year', 'month']).size().reset_index(name='Fatal_crashes').copy()

# dataset contains all collisions
data_crashes = data2.groupby(['year','month']).size().reset_index(name='Crashes').copy()

# Merge datasets
data_killed_crashes = pd.merge(data_killed, data_crashes, how='left', left_on=['year', 'month'], right_on=['year','month']).copy()

# Calculate frequency
data_killed_crashes['forhold'] = data_killed_crashes['Fatal_crashes']/data_killed_crashes['Crashes']

# year and month to string
data_killed_crashes['year'] = data_killed_crashes['year'].astype(str) 
data_killed_crashes['month'] = data_killed_crashes['month'].astype(str) 

# set index to year,month
data_killed_crashes.set_index(['year', 'month'], inplace=True)

#anders data_killed_crashes2 = data_killed_crashes.reset_index()

#anders data_killed_crashes2['year_month']=list(zip(data_killed_crashes2['year'],data_killed_crashes2['month']))

source_killedcrash = ColumnDataSource(data=data_killed_crashes)
#anders source_killedcrash_real = ColumnDataSource(data_killed_crashes2)

# Strings for figure 
years = ['{}'.format(i) for i in data2['CRASH DATE'].dt.year.sort_values(ascending=True).unique()]
months = ['{}'.format(i) for i in data2['CRASH DATE'].dt.month.sort_values(ascending=True).unique()]
x = [(y, m) for y in years for m in months]

# figure layout
p_fatal = figure(x_range=FactorRange(*x), y_range= (0,0.0025), plot_height=300, plot_width=1000, title="Fatal collisions per month over year",
           toolbar_location=None, x_axis_label = 'month/year', y_axis_label = 'frequency',tools="hover", tooltips="@year_month: @forhold")

# Add data into figure 
p_fatal.line(x='year_month', y='forhold', width=0.9, source=source_killedcrash, legend_label='frequency')

# Horizontal line, vision zero speed action 
p_fatal.line(x= [('2014','8'), ('2014','8')] ,y= [0,0.004], width=0.9, line_color="red", name='Vision zero', legend_label='Vision zero action')

p_fatal.y_range.start = 0
p_fatal.x_range.range_padding = 0
p_fatal.xaxis.major_label_orientation = 'vertical'
#p.xgrid.grid_line_color = None


# Plot Contributing factors

In [58]:
# doing a nested np.where(), default valaue equals to X
data2['timeinterval'] = np.where((data2.hour >= 0) & (data2.hour < 6), '0-4', 
                             np.where((data2.hour >= 5) & (data2.hour < 10), '5-9', 
                                      np.where((data2.hour >= 10) & (data2.hour < 16), '10-14',
                                               np.where((data2.hour >= 15) & (data2.hour < 20), '15-19',
                                                       np.where((data2.hour >= 20) & (data2.hour < 24), '20-23'  ,'X' )))))

In [59]:
######################### Contributing factor #############################
#from collections import Counter

vehicle1 = data2[['CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 1':'CONTRIBUTING'})
vehicle2 = data2[['CONTRIBUTING FACTOR VEHICLE 2', 'COLLISION_ID']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 2':'CONTRIBUTING'})
vehicle3 = data2[['CONTRIBUTING FACTOR VEHICLE 3', 'COLLISION_ID']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 3':'CONTRIBUTING'})
vehicle4 = data2[['CONTRIBUTING FACTOR VEHICLE 4', 'COLLISION_ID']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 4':'CONTRIBUTING'})
vehicle5 = data2[['CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 5':'CONTRIBUTING'})


frames = [vehicle1, vehicle2, vehicle3, vehicle4, vehicle5]
result = pd.concat(frames)

# fjerner alle unspecified
result2 = result.loc[result['CONTRIBUTING'] != 'Unspecified']

# Vælger top 21 med flest ulykker, grænsen går ved 10000 ulykker. Unspecified er sorteret fra 
result3 = result2.groupby('CONTRIBUTING').size().sort_values(ascending=False).head(21).copy()
list_of_contributing  = result3.index.to_list()
list_of_contributing

# definerer nyt datasæt, filtreret med de 21 valgte contributing
data_contributing = data2[(data2['CONTRIBUTING FACTOR VEHICLE 1'].isin(list_of_contributing)) | (data2['CONTRIBUTING FACTOR VEHICLE 2'].isin(list_of_contributing))| (data2['CONTRIBUTING FACTOR VEHICLE 3'].isin(list_of_contributing)) | (data2['CONTRIBUTING FACTOR VEHICLE 4'].isin(list_of_contributing)) | (data2['CONTRIBUTING FACTOR VEHICLE 5'].isin(list_of_contributing))]


In [60]:
contribute1 = data2[['CONTRIBUTING FACTOR VEHICLE 1', 'year', 'month','timeinterval']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 1':'CONTRIBUTING'})
contribute2 = data2[['CONTRIBUTING FACTOR VEHICLE 2', 'year', 'month', 'timeinterval']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 2':'CONTRIBUTING'})
contribute3 = data2[['CONTRIBUTING FACTOR VEHICLE 3', 'year', 'month', 'timeinterval']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 3':'CONTRIBUTING'})
contribute4 = data2[['CONTRIBUTING FACTOR VEHICLE 4', 'year', 'month', 'timeinterval']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 4':'CONTRIBUTING'})
contribute5 = data2[['CONTRIBUTING FACTOR VEHICLE 5', 'year', 'month', 'timeinterval']].rename(columns={'CONTRIBUTING FACTOR VEHICLE 5':'CONTRIBUTING'})


frames_contribute = [contribute1, contribute2, contribute3, contribute4, contribute5]
result_contribute = pd.concat(frames_contribute)
result_contribute_final = result_contribute[result_contribute['CONTRIBUTING'].isin(list_of_contributing)] 
#result_contribute_final

In [61]:
# Oversigt over antallet af collisions over år. Det kunne godt se ud til at det er tilfældigt hvad der noteres eller det faktisk er fordi der er ændringer i trafikken 
m = result_contribute_final.groupby([ 'year', 'month', 'CONTRIBUTING']).size().unstack().reset_index().copy()

# Erstatter alle nan med 0
df_contri = m.fillna(0)

#test1 = m[['year','month','Alcohol Involvement']]

In [62]:
# dataset contains all collisions
data_crashes = data2.groupby(['year','month']).size().reset_index(name='Crashes').copy()


In [63]:
# Merge datasets
data_contri_crashes = pd.merge(df_contri, data_crashes, how='left', left_on=['year', 'month'], right_on=['year','month']).copy()

# year and month to string
data_contri_crashes['year'] = data_contri_crashes['year'].astype(str) 
data_contri_crashes['month'] = data_contri_crashes['month'].astype(str) 

# set index to year,month
data_contri_crashes.set_index(['year', 'month'], inplace=True)

# frequency contributing A (year yyyy, month x) / number of crashes (year yyyy, month x)
data_contri_crashes2 = data_contri_crashes.loc[:, 'Alcohol Involvement':'View Obstructed/Limited'].div(data_contri_crashes['Crashes'], axis=0).copy()

#data_contri_crashes2
#data_contri_crashes

In [64]:
new_data_1 = data_contri_crashes2.reset_index()

In [65]:
new_data_1['year_month']=list(zip(new_data_1['year'],new_data_1['month']))

In [66]:
test = data_contri_crashes2[['Alcohol Involvement']].rename(columns={'Alcohol Involvement':'forhold_contri'}).copy()

source_contricrash = ColumnDataSource(data=test)
real_source_select = ColumnDataSource(new_data_1)


#print(source_contricrash.data)

In [67]:
# figure layout
p = figure(x_range=FactorRange(*x), y_range= (0,0.02), plot_height=300, plot_width=1000, title="Overskrift",
           toolbar_location=None, x_axis_label = 'month/year', y_axis_label = 'frequency',tools="hover", tooltips="@year_month: @forhold_contri")

# Add data into figure 
p.line(x='year_month', y='forhold_contri', width=0.9, source=source_contricrash, line_color="green", legend_label='frequency')

# Horizontal line, vision zero speed action 
p.line(x= [('2014','8'), ('2014','8')] ,y= [0,1], width=0.9, line_color="red", name='Vision zero', legend_label='Vision zero action')

p.y_range.start = 0
p.x_range.range_padding = 0
p.xaxis.major_label_orientation = 'vertical'
#p.xgrid.grid_line_color = None

# Correlations table

In [68]:
# Oversigt over antallet af collisions over år. Det kunne godt se ud til at det er tilfældigt hvad der noteres eller det faktisk er fordi der er ændringer i trafikken 
m2 = result_contribute_final.groupby(['year','CONTRIBUTING']).size().unstack().reset_index().copy()

# Erstatter alle nan med 0
df_contri2 = m2.fillna(0)
#m2

In [69]:
timeinterval_data = data2[['year', 'timeinterval']].copy()

timeinterval_data2 = timeinterval_data.groupby(['year', 'timeinterval']).size().unstack().reset_index().copy()

In [70]:
# Merge datasets
data_timeinterval = pd.merge(m2, timeinterval_data2, how='left', left_on=['year'], right_on=['year']).copy()
#data_timeinterval

In [71]:
pearsoncorr = data_timeinterval.corr(method='pearson')
list_columns = pearsoncorr.columns.to_list

In [72]:

pearsoncorr_plot = pearsoncorr.loc[['Alcohol Involvement', 'Backing Unsafely','Driver Inattention/Distraction', 'Driver Inexperience',
       'Failure to Yield Right-of-Way', 'Fatigued/Drowsy',
       'Following Too Closely', 'Lost Consciousness', 'Other Vehicular',
       'Outside Car Distraction', 'Oversized Vehicle', 'Passing Too Closely',
       'Passing or Lane Usage Improper', 'Pavement Slippery',
       'Prescription Medication', 'Reaction to Uninvolved Vehicle',
       'Traffic Control Disregarded', 'Turning Improperly',
       'Unsafe Lane Changing', 'Unsafe Speed', 'View Obstructed/Limited'], ['0-4', '5-9', '10-14', '15-19','20-23']]

#plt.figure(figsize = (10,8))
#ax = sb.heatmap(pearsoncorr_plot, 
 #           cmap='RdBu_r',
 #           annot=True)

In [73]:
from bokeh.io import show, output_notebook
import matplotlib.cm as cm
import matplotlib as mpl
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    NumeralTickFormatter,
    ColorBar,
    FactorRange
)
from bokeh.plotting import figure
from bokeh.palettes import BuPu
import holoviews as hv #There is a reason we have to do this here but its not important. Holoviews is the next library
#hv.extension('bokeh')

#output_notebook()
# Bokeh does not have its own gradient color maps supported but you can easily use on from matplotlib.
colormap =cm.get_cmap("BuPu")
bokehpalette = [mpl.colors.rgb2hex(m) for m in colormap(np.arange(colormap.N))]

#this mapper is what transposes a numerical value to a color. 
mapper = LinearColorMapper(palette=bokehpalette, low=pearsoncorr.min().min(), high=pearsoncorr.max().max())
x_range = FactorRange(factors=pearsoncorr.columns.astype('str'))


timeinterval = list(pearsoncorr_plot.columns.astype('str'))
contribution = list(pearsoncorr_plot.index)

pearson_tabel2 = pearsoncorr_plot.unstack().reset_index(name='antal')
pearson_tabel22 = pearsoncorr_plot.unstack().reset_index(name='antal')
source_corr = ColumnDataSource(pearson_tabel2)

z = figure(title="Correlation between timeinterval and contribution", x_range=timeinterval, y_range=contribution
           , toolbar_location='below', toolbar_sticky=False, tools="hover", tooltips="Correlation: @antal")

z.rect(x='level_0', y='level_1', width=1, height=1, source=source_corr
       , fill_color={'field': 'antal', 'transform': mapper}, line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="6pt",
                     ticker=BasicTicker(desired_num_ticks=8),
                     formatter=NumeralTickFormatter(format="0.0"),
                      border_line_color=None, location=(0, 0),label_standoff=6)

z.add_layout(color_bar, 'right')
z.xaxis.axis_label = 'Timeinterval'
z.yaxis.axis_label = 'Contribution'

In [74]:
#output_file("select.html")

# Liste over alle columns, man skal kunne vælge imellem:
select_list = list(data_contri_crashes2.columns.values)
#select_list = select_list + ['All correlations']

select = Select(title="Contribution:", value='Alcohol Involvement', options=select_list)
slider = Slider(start=2013, end=2019, value=2013, step=1, title= 'Year')




callback = CustomJS(args={'real_source_select':real_source_select,'source_contricrash':source_contricrash,'select':select,'prng':p.y_range},
                    code = """
                var data_source = real_source_select.data;
                var data_to_replace = [];
                var select_value = select.value;
                data_to_replace['forhold_contri'] = data_source[select_value].slice(0);
                data_to_replace['year_month'] = data_source['year_month'].slice(0);
                
                prng.end = Math.max(...data_to_replace['forhold_contri']);
                
                source_contricrash.data = data_to_replace              
                source_contricrash.change.emit();

""")

callback2 = CustomJS(args={'source_p21_real':source_p21_real,'source_p21':source_p21, 'source_p22_real':source_p22_real, 'source_p22':source_p22, 'source_p12_real':source_p12_real, 'source_p12':source_p12, 'slider':slider},
                    code = """
              
                var slival = slider.value;
                var data_source_p21 = source_p21_real.data;
                var data_source_p22 = source_p22_real.data;
                var data_source_p12 = source_p12_real.data;
                
                var data_to_replace_p21 = [];
                data_to_replace_p21['year'] = [];
                data_to_replace_p21['weekday'] = [];
                data_to_replace_p21['BRONX'] = [];
                data_to_replace_p21['BROOKLYN'] = [];
                data_to_replace_p21['MANHATTAN'] = [];
                data_to_replace_p21['QUEENS'] = [];
                data_to_replace_p21['STATEN ISLAND'] = [];
                data_to_replace_p21['UNKNOWN'] = [];
                
                var data_to_replace_p22 = [];
                data_to_replace_p22['year'] = [];
                data_to_replace_p22['hour'] = [];
                data_to_replace_p22['BRONX'] = [];
                data_to_replace_p22['BROOKLYN'] = [];
                data_to_replace_p22['MANHATTAN'] = [];
                data_to_replace_p22['QUEENS'] = [];
                data_to_replace_p22['STATEN ISLAND'] = [];
                data_to_replace_p22['UNKNOWN'] = [];
                
                var data_to_replace_p12 = [];
                data_to_replace_p12['year'] = [];
                data_to_replace_p12['month'] = [];
                data_to_replace_p12['BRONX'] = [];
                data_to_replace_p12['BROOKLYN'] = [];
                data_to_replace_p12['MANHATTAN'] = [];
                data_to_replace_p12['QUEENS'] = [];
                data_to_replace_p12['STATEN ISLAND'] = [];
                data_to_replace_p12['UNKNOWN'] = [];

                for(var i=0;i<data_source_p21['year'].length;i++){
                    if(data_source_p21["year"][i] == slival) {
                        data_to_replace_p21['year'].push(data_source_p21['year'][i]);
                        data_to_replace_p21['weekday'].push(data_source_p21['weekday'][i]);
                        data_to_replace_p21['BRONX'].push(data_source_p21['BRONX'][i]);
                        data_to_replace_p21['BROOKLYN'].push(data_source_p21['BROOKLYN'][i]);
                        data_to_replace_p21['MANHATTAN'].push(data_source_p21['MANHATTAN'][i]);
                        data_to_replace_p21['QUEENS'].push(data_source_p21['QUEENS'][i]);
                        data_to_replace_p21['STATEN ISLAND'].push(data_source_p21['STATEN ISLAND'][i]);
                        data_to_replace_p21['UNKNOWN'].push(data_source_p21['UNKNOWN'][i]);
                    }
                }
                
                for(var i=0;i<data_source_p22['year'].length;i++){                    
                    if(data_source_p22["year"][i] == slival) {
                        data_to_replace_p22['year'].push(data_source_p22['year'][i]);
                        data_to_replace_p22['hour'].push(data_source_p22['hour'][i]);
                        data_to_replace_p22['BRONX'].push(data_source_p22['BRONX'][i]);
                        data_to_replace_p22['BROOKLYN'].push(data_source_p22['BROOKLYN'][i]);
                        data_to_replace_p22['MANHATTAN'].push(data_source_p22['MANHATTAN'][i]);
                        data_to_replace_p22['QUEENS'].push(data_source_p22['QUEENS'][i]);
                        data_to_replace_p22['STATEN ISLAND'].push(data_source_p22['STATEN ISLAND'][i]);
                        data_to_replace_p22['UNKNOWN'].push(data_source_p22['UNKNOWN'][i]);
                    }
                }
                
                for(var i=0;i<data_source_p12['year'].length;i++){                     
                    if(data_source_p12["year"][i] == slival) {
                        data_to_replace_p12['year'].push(data_source_p12['year'][i]);
                        data_to_replace_p12['month'].push(data_source_p12['month'][i]);
                        data_to_replace_p12['BRONX'].push(data_source_p12['BRONX'][i]);
                        data_to_replace_p12['BROOKLYN'].push(data_source_p12['BROOKLYN'][i]);
                        data_to_replace_p12['MANHATTAN'].push(data_source_p12['MANHATTAN'][i]);
                        data_to_replace_p12['QUEENS'].push(data_source_p12['QUEENS'][i]);
                        data_to_replace_p12['STATEN ISLAND'].push(data_source_p12['STATEN ISLAND'][i]);
                        data_to_replace_p12['UNKNOWN'].push(data_source_p12['UNKNOWN'][i]);

                    }
                }
                
                
                source_p21.data = data_to_replace_p21;
                source_p22.data = data_to_replace_p22;
                source_p12.data = data_to_replace_p12;
                source_p21.change.emit();
                source_p22.change.emit();
                source_p12.change.emit();
                

""")



# def update_plot(attr, old, new):
#     #if select.value == 'All correlations':
#         #source_corr.data = pearson_tabel22.copy()
#     if select.value=='Alcohol Involvement':
#         source_contricrash.data = test.copy()
#         p.y_range.end = data_contri_crashes2[select.value].max()
#     else:
#         source_contricrash.data = data_contri_crashes2[[select.value]].rename(columns={select.value:'forhold_contri'}).copy()
#         p.y_range.end = data_contri_crashes2[select.value].max()
#         #source_corr.data = pearson_tabel2.loc[pearson_tabel2['level_1'] == select.value]
  
#     yr=slider.value
#     source_p21.data = data2.loc[data2['year'] == yr].groupby(['year', 'weekday','BOROUGH']).size().unstack().reset_index().copy()
#     source_p12.data = data2.loc[data2['year'] == yr].groupby(['month','BOROUGH']).size().unstack().copy()
#     source_p22.data = data2.loc[data2['year']==yr].groupby(['hour','BOROUGH']).size().unstack().copy()
    

slider.js_on_change('value', callback2)    
layout = row(column(div_text, p_11, p_21), column(slider,p_12, p_22))
tab1 = Panel(child = layout, title = 'Overview')

select.js_on_change('value', callback)
layout3 = column(row(select, width=400), p)

layout2 = row(column(layout3, p_fatal), column(z), sizing_mode="scale_width")
tab2 = Panel(child = layout2, title = 'Contributing and fatal collisions')

In [77]:
show(row(p_21,slider))

In [37]:
tabs = Tabs(tabs = [tab1,tab2,tab3,tab4])

In [38]:
#show(select)

#curdoc().add_root(tabs)

show(tabs)
output_file("js_events.html", title="Fuck2")